In [1]:
import requests
from bs4 import BeautifulSoup
import wget
from PIL import Image
import os

In [2]:
# Ruta donde se deben eliminar las carpetas
output_path = 'img_download'

# Eliminar todas las carpetas en output_path
if os.path.exists(output_path):
    # Iterar sobre los elementos dentro del directorio
    for item in os.listdir(output_path):
        item_path = os.path.join(output_path, item)
        # Eliminar archivos
        if os.path.isfile(item_path):
            os.remove(item_path)
        # Eliminar carpetas y su contenido de forma recursiva
        elif os.path.isdir(item_path):
            for root, dirs, files in os.walk(item_path, topdown=False):
                for name in files:
                    os.remove(os.path.join(root, name))
                for name in dirs:
                    os.rmdir(os.path.join(root, name))
            os.rmdir(item_path)  # Finalmente eliminar la carpeta vacía
else:
    print(f"The folder {output_path} does not exist.")

In [3]:
# Asegurarse de que la carpeta exista después de eliminar su contenido
os.makedirs(output_path, exist_ok=True)

# Ruta de la carpeta de entrada con los archivos .txt
folder_path = '/mnt/D8E84E4DE84E2A58/Env_python/Create_video_news/0_create text/text/en/final/'

# Lista de archivos .txt en la carpeta
files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Ordenar los archivos si es necesario
files.sort()

In [4]:
def descargar_imagenes_bing(query, num_imagenes, carpeta_destino):
    # Genera la URL de búsqueda de Bing con filtro de tamaño grande
    url = f"https://www.bing.com/images/search?q={query}&qft=+filterui:imagesize-medium&form=IRFLTR&first=1"

    # Realiza la solicitud GET al servidor
    response = requests.get(url)
    if response.status_code != 200:
        print("Error al realizar la solicitud a Bing.")
        return

    # Procesar el HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra las URLs de las imágenes en el HTML
    imagenes = soup.find_all('a', class_='iusc')

    # Crea una carpeta de destino si no existe
    if not os.path.exists(carpeta_destino):
        os.makedirs(carpeta_destino)

    # Descargar las imágenes
    contador = 0
    for img in imagenes:
        if contador >= num_imagenes:
            break

        # Extraer datos JSON que contienen la URL de la imagen completa
        try:
            m = img.get('m')
            if m:
                # La URL de alta resolución está en el atributo `m` en formato JSON
                json_data = eval(m)
                img_url = json_data.get('murl')

                if img_url:
                    # Descargar y guardar la imagen en la carpeta de destino
                    print(f"Descargando imagen {contador + 1} de {num_imagenes}: {img_url}")
                    img_nombre = os.path.join(carpeta_destino, f"{query}_{contador + 1}.jpg")
                    wget.download(img_url, img_nombre)
                    contador += 1

        except Exception as e:
            print(f"Error al procesar la imagen: {e}")

    print(f"\nDescarga completada. {contador} imágenes guardadas en '{carpeta_destino}'.")


In [5]:
input_thematic = str(input('introduzca la tematica de la image:'))

In [6]:
# Uso del script
consulta = input_thematic
numero_de_imagenes = 5
descargar_imagenes_bing(consulta, numero_de_imagenes, carpeta_destino='img_download')

Descargando imagen 1 de 5: http://www.nattapon.com/wp-content/uploads/2011/07/22251_linux.jpg
Descargando imagen 2 de 5: https://areatecnologia.com/informatica/imagenes/linux.jpg
Descargando imagen 3 de 5: https://blog.internxt.com/es/content/images/2022/07/Linux.png
Descargando imagen 4 de 5: https://www.geekno.com/wp-content/uploads/2018/09/razones-por-las-que-utilzar-linux.jpg
Error al procesar la imagen: HTTP Error 403: Forbidden
Descargando imagen 4 de 5: https://www.analyticssteps.com/backend/media/thumbnail/3033404/5694625_1665050003_Linux OSArtboard 1.jpg
Descargando imagen 5 de 5: https://www.softzone.es/app/uploads-softzone.es/2022/08/Portatil-Linux.jpg

Descarga completada. 5 imágenes guardadas en 'img_download'.
